https://colab.research.google.com/drive/1xcQWHsQUegRIRWYYUOCO66HOt0Y6qeHL?usp=sharing

In [ ]:
import torch
!nvidia-smi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.memory_stats()

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



OrderedDict()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
from  datetime import datetime

np.random.seed(42)

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor, Lambda, Compose
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

torch.manual_seed(17)

In [ ]:
def setup():
  from google.colab import drive
  #drive.flush_and_unmount()
  drive.mount('/content/gdrive')

  os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle/ML_Assignment4"

  %cd /content/gdrive/My Drive/Kaggle/ML_Assignment4
  path = %pwd
  PATH_tr = os.path.join(path, 'savetr_vgg.pt')
  PATH_te = os.path.join(path, 'savete_vgg.pt')

  return PATH_tr, PATH_te

In [ ]:
def no_transforms(datadir, valid_size = .2):
    train_transforms = transforms.Compose([
 transforms.Resize(256),                   
 transforms.CenterCrop(224),               
 transforms.ToTensor(),                    
 transforms.Normalize(                     
 mean=[0.485, 0.456, 0.406],               
 std=[0.229, 0.224, 0.225]                 
 )])
    
    test_transforms = transforms.Compose([
 transforms.Resize(256),                   
 transforms.CenterCrop(224),               
 transforms.ToTensor()])
 
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size, num_workers = 2)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size, num_workers = 2)
    return len(train_idx), len(test_idx), trainloader, testloader

In [ ]:
def augementated_data(datadir, valid_size = .2):
    train_transforms = transforms.Compose([
 transforms.Resize(256),                   
 transforms.RandomCrop(224),
 transforms.RandomHorizontalFlip(p=0.5),
 transforms.RandomVerticalFlip(p=0.5),    
 transforms.RandomApply(torch.nn.ModuleList([
    transforms.ColorJitter(),
    transforms.RandomAutocontrast(p=0.5)
 ]), p=0.3),           
 transforms.ToTensor(),                    
 transforms.Normalize(                     
 mean=[0.485, 0.456, 0.406],               
 std=[0.229, 0.224, 0.225]                 
 )])
    
    test_transforms = transforms.Compose([
 transforms.Resize(256),                   
 transforms.CenterCrop(224),               
 transforms.ToTensor()])
 
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    
    #for i in train_data:
    print(train_data)

    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size, num_workers = 2)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size, num_workers = 2)
    return len(train_idx), len(test_idx), trainloader, testloader

In [ ]:
def imshow(image, ax=None, title=None, normalize=True):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    image = image.numpy().transpose((1, 2, 0))

    if normalize:
        mean = np.array([0.5, 0.5, 0.5])
        std = np.array([0.5, 0.5, 0.5])
        image = std * image + mean
        image = np.clip(image, 0, 1)

    ax.imshow(image)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_xticklabels('')
    ax.set_yticklabels('')

    return ax

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def get_model():
  vgg19 = torchvision.models.vgg11(pretrained = True, progress = True)
  #vgg19.eval()
  vgg19.to(device)
  for param in vgg19.parameters():
    param.requires_grad = False
  number_features = vgg19.classifier[-1].in_features
  features = list(vgg19.classifier.children())[:-1] # Remove last layer
  features.extend([torch.nn.Linear(number_features, 1024)])
  features.extend([torch.nn.ReLU(), nn.Dropout(0.2)])
  features.extend([torch.nn.Linear(1024, len(classes))])
  vgg19.classifier = torch.nn.Sequential(*features)

  vgg19 = vgg19.to(device)

  return vgg19

In [ ]:
def hyperparameters(model):
  criterion = nn.CrossEntropyLoss()
  from torch.optim import lr_scheduler

  # Observe that all parameters are being optimized
  #optimizer_ft = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

  optimizer_ft = optim.Adagrad(model.parameters(), lr=learning_rate, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10)

  # Decay LR by a factor of 0.1 every 7 epochs
  exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=step_size, gamma=gamma)

  return optimizer_ft,exp_lr_scheduler,criterion

In [ ]:
def train_model(model, criterion, optimizer, scheduler, PATH_tr, PATH_te, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    epoch = -1
    lossl = []
    accl = []
    te_lossl = []
    te_accl = []

    #model, optimizer_ft, epoch, te_lossl, te_accl = load(PATH_te, model, optimizer)
    #model, optimizer_ft, epoch, lossl, accl = load(PATH_tr, model, optimizer)
    

    #epoch = 1
    for epoch in range(epoch+1, num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        #print(torch.cuda.memory_summary(abbreviated=True))
        print('-' * 10)

        now = datetime.now().isoformat()
        #current_time = now.strftime("%H:%M:%S")
        print(now)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            #print('1')
            running_loss = 0.0
            running_corrects = 0

            batch=0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                batch += 1
                #print(labels)
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                batch_loss = (running_loss*batch*batch_size) / dataset_sizes[phase]
                batch_acc = (running_corrects.double()*batch*batch_size) / dataset_sizes[phase]
            
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print()
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == 'train':
              lossl.append(epoch_loss)
              accl.append(epoch_acc)
              torch.save({
              'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': lossl,
              'acc': accl
              },PATH_tr)

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def load(PATH, model, optimizer):  
  checkpoint = torch.load(PATH, map_location=torch.device('cpu'))
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']
  epoch_loss = checkpoint['loss']
  epoch_acc = checkpoint['acc']
  return model, optimizer, epoch, epoch_loss, epoch_acc

In [ ]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.cpu().numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

###No_Augementation VGG11 Transfer Learning

In [ ]:
import copy

classes = ['Articuno', 'Dragonite']
batch_size = 4
num_epochs = 10
learning_rate = 0.01
step_size = 10
gamma = 0.1
momentum = 0.8
PATH_tr, PATH_te = setup()
#writer = SummaryWriter('/content/gdrive/My Drive/Kaggle/ML_Assignment4/runs/')
dataloaders = dict.fromkeys(['train', 'test'])
train_len, test_len, dataloaders['train'], dataloaders['test'] = no_transforms('train/', .25)
dataset_sizes = {'train':train_len, 'test':test_len}
print(dataset_sizes)
#images, labels = next(iter(dataloaders['train']))
#imshow(images[5], normalize=True);
model = get_model()
optimizer_ft, exp_lr_scheduler, criterion = hyperparameters(model)


#model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, PATH_tr, PATH_te, num_epochs=num_epochs)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Kaggle/ML_Assignment4
{'train': 55, 'test': 18}


Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

###Augementation using transforms VGG Transfer Learning

In [ ]:
import copy

classes = ['Articuno', 'Dragonite']
batch_size = 4
num_epochs = 10
learning_rate = 0.01
step_size = 10
gamma = 0.1
momentum = 0.8
PATH_tr, PATH_te = setup()
writer = SummaryWriter('/content/gdrive/My Drive/Kaggle/ML_Assignment4/runs/')
dataloaders = dict.fromkeys(['train', 'test'])
train_len, test_len, dataloaders['train'], dataloaders['test'] = augementated_data('train/', .25)
dataset_sizes = {'train':train_len, 'test':test_len}
print(dataset_sizes)
#images, labels = next(iter(dataloaders['train']))
#imshow(images[5], normalize=True);
model = get_model()
optimizer_ft, exp_lr_scheduler, criterion = hyperparameters(model)


model_ft = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, PATH_tr, PATH_te, num_epochs=num_epochs)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Kaggle/ML_Assignment4
Dataset ImageFolder
    Number of datapoints: 73
    Root location: train/
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               RandomCrop(size=(224, 224), padding=None)
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               RandomApply(
               p=0.3
               ColorJitter(brightness=None, contrast=None, saturation=None, hue=None)
               RandomAutocontrast(p=0.5)
           )
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
{'train': 55, 'test': 18}
Epoch 0/9
----------
2022-04-15T06:09:24.939883

train Loss: 4.4808 Acc: 0.7273

test Loss: 0.5227 Acc: 0.8889

Epoch 1/9
----------
2022-04-15

In [ ]:
model, optimizer_ft, epoch, lossl, accl = load(PATH_tr, model, optimizer_ft)
